# L3: Agentic Sales Pipeline

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Load API tokens for our 3rd party APIs

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [4]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [5]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [6]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [7]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2025-05-14 17:03:02,019 - 140239370890112 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [8]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [9]:
flow.plot()

Graph saved as crewai_flow_graph.html


In [10]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [11]:
emails = await flow.kickoff()

2025-05-14 17:03:02,338 - 140239370890112 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int



# Agent: Lead Data Specialist
## Thought: Thought: I have gathered substantial information about João Moura's professional background, highlighting his career journey, leadership experience, and contributions at Clearbit. Now, I will gather information about Clearbit's industry, company size, revenue, and market presence.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company profile industry company size revenue market presence\"}"
## Tool Output: 

Search results: Title: Leverage 100+ business data attributes - Clearbit
Link: https://clearbit.com/attributes
Snippet: Enrich your CRM and database, manage leads efficiently, and enable marketing personalization with over 100 business data points.
---
Title: Clearbit - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/clearbit
Snippet: Clearbit develops business intelligence application program interface tools designed to help companies to find information about their c



# Agent: Lead Data Specialist
## Final Answer: 
### Comprehensive Data Report

#### Personal Information:
- **Name**: João Moura
- **Job Title**: Director of Engineering
- **Role Relevance**: 9/10 (as a Director of Engineering, he has significant influence over technical decisions and initiatives)
- **Professional Background**: 
  João Moura has over 20 years of experience in software engineering. He began programming at a young age and has held various leadership roles, including a significant tenure as the Senior Engineering Manager at Clearbit, where he built and scaled the engineering team. His career includes a background in law, which he transitioned away from to focus on technology, culminating in his current position where he leads the engineering efforts for a major data activation platform.

#### Company Information:
- **Company Name**: Clearbit
- **Industry**: Software, Technology, Business Intelligence, Data and Analytics, Developer APIs
- **Company Size**: 31 employees
-



# Agent: Cultural Fit Analyst
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://clearbit.com/blog/company-values\"}"
## Tool Output: 
Defining Clearbit's company valuesClearbit customer loginBlog>Defining Clearbit's company valuesDefining Clearbit's company valuesMatt Sornson|August 12, 2019|4 minute readIn mid-2018, Clearbit was hitting an inflection point.
Since our founding in 2014, we'd been running as a small, hyper-efficient, and profitable business. Just about everyone in the company would weigh in on new hires, and every new employee spent significant time with the entire leadership team to understand what our values were. This organic model started to show some cracks as we started to scale.
At thirty-odd employees, Clearbit did already have a real set of values, even if not everyone could verbalize them. In line with how we were building the company — deliberately and with a close eye on creating a culture we all wanted to be a part of — it wa



# Agent: Cultural Fit Analyst
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.tealhq.com/company/clearbit\"}"
## Tool Output: 
Attention Required! | Cloudflare
Please enable cookies.
Sorry, you have been blocked
You are unable to access tealhq.com
Why have I been blocked?
This website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data.
What can I do to resolve this?
You can email the site owner to let them know you were blocked. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page.
Cloudflare Ray ID: 93fbf8246fd5ed3b
•
 Your IP:
 Click to reveal
52.52.221.170
•
Performance & security by Cloudflare


# Agent: Cultural Fit Analyst
## Using tool: Read website content
## Tool In



# Agent: Lead Scorer and Validator
## Thought: I need to evaluate the lead data provided and aggregate it against the predefined scoring criteria for the calculated lead score. I'll start by gathering additional data on the company, Clearbit, to assess its size and market presence, which are key factors in the scoring process.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company size and market presence information\"}"
## Tool Output: 

Search results: Title: Leverage 100+ business data attributes - Clearbit
Link: https://clearbit.com/attributes
Snippet: Enrich your CRM and database, manage leads efficiently, and enable marketing personalization with over 100 business data points.
---
Title: Fit: firmographic and employment data - Clearbit
Link: https://clearbit.com/resources/books/b2b-data/firmographic-demographic
Snippet: Firmographic data: company information, like size, type, industry, revenue, etc., indicating whether the company is a good fi



# Agent: Lead Scorer and Validator
## Final Answer: 
```json
{
  "final_lead_score": 91,
  "scoring_criteria": {
    "Role Relevance": 30,
    "Company Size": 20,
    "Market Presence": 23,
    "Cultural Fit": 18
  },
  "validation_notes": "The evaluation confirms João Moura's alignment with CrewAI's objectives, and Clearbit's size and presence in the market being conducive to our targeted ICP."
}
```




2025-05-14 17:04:03,272 - 140239370890112 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Email Content Writer
## Task: Craft a highly personalized email using the lead's name, job title, company information, and any relevant personal or company achievements. The email should speak directly to the lead's interests and the needs of their company. This is not as cold outreach as it is a follow up to a lead form, so keep it short and to the point. Don't use any salutations or closing remarks, nor too complex sentences.
Our Company and Product: - Company Name: CrewAI - Product: Multi-Agent Orchestration Platform - ICP: Enterprise companies looking into Agentic automation. - Pitch: We are a platform that allows you to orchestrate AI Agents for automations to any vertical.
Use the following information: Personal Info: {'name': 'João Moura', 'job_title': '', 'role_relevance': 10, 'professional_background': None} Company Info: {'company_name': 'Clearbit', 'industry': '', 'company_size': 20, 'revenue': None, 'market_presence': 10} Lead Score: {'score': 91, 'scoring_criteria

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [21]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

KeyError: 'score_crews_results'

In [22]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0003


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,1735,1369,366,3


## Inspecting Results

In [23]:
scores = flow.state["score_crews_results"]

KeyError: 'score_crews_results'

In [24]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,João Moura
Job Title,
Role Relevance,10
Professional Background,None
Company Name,Clearbit
Industry,
Company Size,20
Revenue,None
Market Presence,10
Lead Score,91


## Results

In [25]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

CrewAI’s Multi-Agent Orchestration Platform is designed to elevate your team’s
efficiency at Clearbit by enabling tailored automations across any vertical.
Imagine streamlining your processes while maximizing productivity with AI Agents
orchestrated precisely for your needs.    Ready to unlock this potential?
Schedule a quick 15-minute meeting here [insert scheduling link] to discover how
we can integrate this into your operations.    Not interested in a meeting just
yet? Check out this brief demo [insert demo link] showcasing our platform in
action. Don’t miss the chance to enhance your capabilities—act now!


## How Complex Can it Get?

In [26]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [27]:
flow = SalesPipeline()
flow.plot()

Graph saved as crewai_flow_graph.html


In [28]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)